In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/')
!ls

In [ ]:
import os
os.chdir('/content/drive/My Drive/Datewise-Tweets-Data/May-10-2020')
!ls

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from nltk.stem import WordNetLemmatizer


import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
 yelp = pd.read_csv('RestaurantReviewDataYelp.csv')

In [ ]:
#creating fucntion for lebelling data set for training purpose.
def new_col(row):
    if row['stars'] == 1 or row['stars'] ==2:
        val = 'negative'
    elif row['stars']==4 or row['stars']==5:
        val = 'positive'
    else:
        val = 'neutral'
    return val

In [ ]:
yelp['sentiment'] = yelp.apply(new_col, axis=1)

In [ ]:
yelp.sentiment.value_counts()

positive    30133
negative     9981
neutral      7381
Name: sentiment, dtype: int64

In [ ]:
#cleaning text function.................................
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
lemmatizer = WordNetLemmatizer()
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@#'

# cleaning master function
#this function will handle links with slashes and without slashes..................
def clean_text(text, bigrams=False):
    text = text.lower() # lower case
    text = re.sub('['+my_punctuation + ']+', ' ', text) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    text_token_list = [word for word in text.split(' ')
                       if word not in all_stopwords] # remove stopwords

    text_token_list = [word_rooter(word) if '#' not in word else word
                        for word in text_token_list] # apply word rooter
    
    text_token_list = [lemmatizer.lemmatize(word) if '#' not in word else word
                        for word in text_token_list] # apply lemmatizing
                    
    
    text = ' '.join(text_token_list)
    return text  

In [ ]:
yelp['cleaned_text']=yelp['text'].apply(clean_text)

In [ ]:
yelp['cleaned_text'][400]

'taco burrito quesadilla aren t thing go mr amigo wait yup restaur run asian offer french food bit touch food authent food certainli bring nnstart servic pretti standard time serv littl kid tri polit thing got quickli awkward certain way regardless food care cook surpris tender meat cook certainli talent isn t afraid nnmr amigo best restaur brossard opinion live nearbi haven t favor treat meal mr amigo '

In [ ]:
X_yelp = yelp["cleaned_text"]
y_yelp = yelp["sentiment"]

Last time we used only single word features in our model, which we call 1-grams or unigrams. We can potentially add more predictive power to our model by adding two or three word sequences (bigrams or trigrams) as well. For example, if a review had the three word sequence “didn’t love food” we would only consider these words individually with a unigram-only model and probably not capture that this is actually a negative sentiment because the word ‘love’ by itself is going to be highly correlated with a positive review.
The scikit-learn library makes this really easy to play around with. Just use the ngram_range argument with any of the ‘Vectorizer’ classes.

Instead of simply noting whether a word appears in the review or not, we can include the number of times a given word appears. This can give our sentiment classifier a lot more predictive power. For example, if a movie reviewer says ‘amazing’ or ‘terrible’ multiple times in a review it is considerably more probable that the review is positive or negative, respectively.

In [ ]:
#unigram+bigram+existence of word
from sklearn.feature_extraction.text import CountVectorizer
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(X_yelp)
X_ngram_t = ngram_vectorizer.transform(X_yelp)



In [ ]:
print('Shape of Sparse Matrix: ', X_ngram_t.shape)

Shape of Sparse Matrix:  (47495, 990888)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ngram_t,y_yelp,test_size =.10)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
       lr = LogisticRegression(C=c)
       lr.fit(X_train, y_train)
#kfold = KFold(n_splits=10, random_state=SEED)
#cv_results1 = cross_val_score(lr, X_train, y_train, cv = kfold, scoring=scoring[0])
        
       print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.8010526315789473
Accuracy for C=0.05: 0.8143157894736842
Accuracy for C=0.25: 0.8115789473684211
Accuracy for C=0.5: 0.8107368421052632
Accuracy for C=1: 0.8096842105263158


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
accuracy_score(y_test, nb.predict(X_test))

0.7690526315789473

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))

Accuracy for C=0.01: 0.8149473684210526
Accuracy for C=0.05: 0.8136842105263158
Accuracy for C=0.25: 0.808
Accuracy for C=0.5: 0.8061052631578948
Accuracy for C=1: 0.8027368421052632


In [ ]:
# bigram+unigram+frequency
#unigram+bigram+existence of word
from sklearn.feature_extraction.text import CountVectorizer
ngram_vectorizer = CountVectorizer(binary=False, ngram_range=(1, 2))
ngram_vectorizer.fit(X_yelp)
X_ngram_f = ngram_vectorizer.transform(X_yelp)

In [ ]:
print('Shape of Sparse Matrix: ', X_ngram_f.shape)

Shape of Sparse Matrix:  (47495, 990888)


In [ ]:
from sklearn.model_selection import train_test_split

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_ngram_f,y_yelp,test_size =.10)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
       lr = LogisticRegression(C=c)
       lr.fit(X_train_f, y_train_f)
#kfold = KFold(n_splits=10, random_state=SEED)
#cv_results1 = cross_val_score(lr, X_train, y_train, cv = kfold, scoring=scoring[0])
        
       print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test_f, lr.predict(X_test_f))))

Accuracy for C=0.01: 0.8016842105263158
Accuracy for C=0.05: 0.8138947368421052
Accuracy for C=0.25: 0.8164210526315789
Accuracy for C=0.5: 0.8134736842105263
Accuracy for C=1: 0.8124210526315789


In [ ]:
nb.fit(X_train_f, y_train_f)
accuracy_score(y_test_f, nb.predict(X_test_f))

0.7707368421052632

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train_f, y_train_f)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test_f, svm.predict(X_test_f))))

Accuracy for C=0.01: 0.8065263157894736
Accuracy for C=0.05: 0.8101052631578948
Accuracy for C=0.25: 0.8061052631578948
Accuracy for C=0.5: 0.8025263157894736
Accuracy for C=1: 0.8006315789473685


So far we’ve chosen to represent each review as a very sparse vector (lots of zeros!) with a slot for every unique n-gram in the corpus (minus n-grams that appear too often or not often enough). Linear classifiers typically perform better than other algorithms on data that is represented in this way.
Support Vector Machines (SVM)
Recall that linear classifiers tend to work well on very sparse datasets (like the one we have). Another algorithm that can produce great results with a quick training time are Support Vector Machines with a linear kernel.

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))

Accuracy for C=0.01: 0.7966315789473685
Accuracy for C=0.05: 0.7926315789473685
Accuracy for C=0.25: 0.7861052631578948
Accuracy for C=0.5: 0.7869473684210526
Accuracy for C=1: 0.7877894736842105


Instead of simply noting whether a word appears in the review or not, we can include the number of times a given word appears. This can give our sentiment classifier a lot more predictive power. For example, if a restraunt reviewer says ‘amazing’ or ‘terrible’ multiple times in a review it is considerably more probable that the review is positive or negative, respectively.

In [ ]:
wc_vectorizer = CountVectorizer(binary=False)
wc_vectorizer.fit(X_yelp)
X_word_count = wc_vectorizer.transform(X_yelp)



In [ ]:
X_train_wc, X_test_wc, y_train_wc, y_test_wc = train_test_split(X_word_count,y_yelp,test_size =.10)

running model.............................logistic regression.....

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train_wc, y_train_wc)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test_wc, lr.predict(X_test_wc))))

Accuracy for C=0.01: 0.7938947368421052
Accuracy for C=0.05: 0.7976842105263158
Accuracy for C=0.25: 0.7945263157894736
Accuracy for C=0.5: 0.7903157894736842
Accuracy for C=1: 0.784


In [ ]:
#naive bayes
nb.fit(X_train_wc, y_train_wc)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
accuracy_score(y_test_wc, nb.predict(X_test_wc))

0.7903157894736842

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train_wc, y_train_wc)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test_wc, svm.predict(X_test_wc))))

Accuracy for C=0.01: 0.7924210526315789
Accuracy for C=0.05: 0.7922105263157895
Accuracy for C=0.25: 0.7791578947368422
Accuracy for C=0.5: 0.7730526315789473
Accuracy for C=1: 0.7684210526315789


Another common way to represent each document in a corpus is to use the tf-idf statistic (term frequency-inverse document frequency) for each word, which is a weighting factor that we can use in place of binary or word count representations.
There are several ways to do tf-idf transformation but in a nutshell, tf-idf aims to represent the number of times a given word appears in a document (a movie review in our case) relative to the number of documents in the corpus that the word appears in — where words that appear in many documents have a value closer to zero and words that appear in less documents have values closer to 1.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),binary=False)
tfidf_vectorizer.fit(X_yelp)
X_tfidf_transform = tfidf_vectorizer.transform(X_yelp)

In [ ]:
print('Shape of Sparse Matrix: ', X_tfidf_transform.shape)

Shape of Sparse Matrix:  (47495, 990888)


In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_tfidf_transform,y_yelp,test_size =.10)

In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train_tf, y_train_tf)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test_tf, lr.predict(X_test_tf))))

Accuracy for C=0.01: 0.6341052631578947
Accuracy for C=0.05: 0.6821052631578948
Accuracy for C=0.25: 0.7734736842105263
Accuracy for C=0.5: 0.7924210526315789
Accuracy for C=1: 0.8004210526315789


In [ ]:
#naive bayes
nb.fit(X_train_tf, y_train_tf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
accuracy_score(y_test_tf, nb.predict(X_test_tf))

0.9412631578947368

In [ ]:
SEED=2000
scoring = ['accuracy']
kfold = KFold(n_splits=10, random_state=SEED)
cv_results1 = cross_val_score(nb, X_train_tf, y_train_tf, cv = kfold, scoring=scoring[0])
print(cv_results1.mean())

0.6379460904640324


In [ ]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train_tf, y_train_tf)
    kfold = KFold(n_splits=10, random_state=SEED)
    
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test_tf, svm.predict(X_test_tf))))

Accuracy for C=0.01: 0.6884210526315789
Accuracy for C=0.05: 0.7787368421052632
Accuracy for C=0.25: 0.8027368421052632
Accuracy for C=0.5: 0.8025263157894736
Accuracy for C=1: 0.8050526315789474
